In [1]:
import pandas as pd
import joblib
import json
from functools import partial

from service.app_api.features.extractor import FeatureExtractor
from service.app_api.features.final_processing import CustomColumnTransformer
from tuning.optuna_tuning import OptunaTuner
import optuna
import wandb

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report, f1_score
# Base classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# Meta classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [2]:
try:
    customers, sales = joblib.load('customers.joblib'), joblib.load('sales.joblib')
except:
    customers, sales = pd.read_excel('ucy_eko_data.xlsx', sheet_name='smile_customers'), pd.read_excel('ucy_eko_data.xlsx', sheet_name='smile_sales')
    joblib.dump(customers, 'customers.joblib')
    joblib.dump(sales, 'sales.joblib')

In [3]:
fe = FeatureExtractor(target_month=3, n_purchases=2, perform_split=True, generation_type='continuous', filtering_set='customers', period=60, subperiod=15)
X_train, X_test, y_train, y_test = fe.transform(sales=sales, customers=customers)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/fedorturchenko/.cache/huggingface/token
Login successful
Successfully logged out.
['fuel_qty', 'labour_cards_catalogue_consumables_qty', 'peak_hours_qty', 'usual_hours_qty', 'weekday_qty', 'weekend_qty', 'unleaded_95_qty', 'last_purchase_qty_share']


In [4]:
qty_cols = [col for col in X_train.columns if 'qty' in col]
col_transform = CustomColumnTransformer(
    cols_for_scaling=qty_cols,
    scaling_algo=RobustScaler(),
    cols_for_ohe=None,
    cols_for_winsor=None,
    cols_to_skip=[col for col in X_train.columns if 'qty' in col]
)

In [5]:
X_train_trans = col_transform.fit_transform(X_train)
X_test_trans = col_transform.fit_transform(X_test)

In [6]:
f1_score_average_weighted = partial(f1_score, average='weighted')

In [7]:
def params_to_txt(optuna_object):
    model = optuna_object.model
    model_name = model.__str__()[:model.__str__().find('(')]
    best_params = model.get_params()
    with open(f'{model_name}.txt', 'w') as f:
        f.write(json.dumps(best_params, indent=4))

## SVC base model

In [8]:
# svc_op = OptunaTuner(
#     SVC,
#     accuracy_score,
#     direction='maximize',
#     random_state=571,
#     verbose=False,
#     tol=1
# )
# svc_op.fit(
#     50, X_train_trans, y_train, X_test_trans, y_test,
#     ('C', 'float', 1e-3, 1.0),
#     ('kernel', 'categorical', ['linear', 'poly', 'rbf', 'sigmoid']),
#     ('degree', 'int', 1, 15)
# )

In [9]:
# optuna.visualization.plot_optimization_history(svc_op.study)

In [10]:
# svc_op.model.get_params()

In [11]:
svc_best = SVC(
    verbose=False,
    random_state=571,
    tol=1
)
svc_best.fit(X_train, y_train)
svc_train_preds = svc_best.predict(X_train)
svc_test_preds = svc_best.predict(X_test)

In [12]:
print('Train:')
print(classification_report(y_train, svc_train_preds), '\n')
print('Test:')
print(classification_report(y_test, svc_test_preds))

Train:
              precision    recall  f1-score   support

           0       0.80      0.90      0.85     14415
           1       0.77      0.58      0.66      7882

    accuracy                           0.79     22297
   macro avg       0.78      0.74      0.75     22297
weighted avg       0.79      0.79      0.78     22297
 

Test:
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      4744
           1       0.76      0.57      0.65      2689

    accuracy                           0.78      7433
   macro avg       0.77      0.74      0.75      7433
weighted avg       0.78      0.78      0.77      7433



## KNN base model

In [13]:
knn_op = OptunaTuner(
    KNeighborsClassifier,
    f1_score_average_weighted,
    direction='maximize',
    n_jobs=7
)
knn_op.fit(
    100, X_train, y_train, X_test, y_test,
    ('n_neighbors', 'int', 1, 50)
)

In [14]:
optuna.visualization.plot_optimization_history(knn_op.study)

In [15]:
knn_best = KNeighborsClassifier(**knn_op.model.get_params())
knn_best.fit(X_train, y_train)
knn_train_preds = knn_best.predict(X_train)
knn_test_preds = knn_best.predict(X_test)

In [16]:
print('Train:')
print(classification_report(y_train, knn_train_preds), '\n')
print('Test:')
print(classification_report(y_test, knn_test_preds))

Train:
              precision    recall  f1-score   support

           0       0.81      0.88      0.85     14415
           1       0.75      0.62      0.68      7882

    accuracy                           0.79     22297
   macro avg       0.78      0.75      0.76     22297
weighted avg       0.79      0.79      0.79     22297
 

Test:
              precision    recall  f1-score   support

           0       0.80      0.88      0.84      4744
           1       0.75      0.61      0.67      2689

    accuracy                           0.79      7433
   macro avg       0.77      0.75      0.76      7433
weighted avg       0.78      0.79      0.78      7433



In [17]:
params_to_txt(knn_op)

## Load best Random Forest model

In [18]:
# api = wandb.Api()
# run = api.run(f"{settings.SETTINGS['WANDB_ENTITY']}/Random-Forest-Classifier/qzbhynm9")

In [19]:
# rf_config = json.loads(run.json_config)
# rf_params = {}
# for param in rf_config:
#     rf_params.update({param: rf_config[param]['value']})

In [20]:
rf_op = OptunaTuner(RandomForestClassifier, f1_score_average_weighted, direction='maximize', random_state=571, n_jobs=7)
rf_op.fit(
    300, X_train, y_train, X_test, y_test,
    ('n_estimators', 'int', 10, 200),
    ('max_depth', 'int', 30, 60),
    ('max_features', 'float', 0.05, 1.0, {'step': 0.05}),
    ('max_samples', 'float', 0.05, 1.0, {'step': 0.05}),
    ('min_samples_leaf', 'float', 1e-4, 1e-2, {'log': True}),
    ('min_samples_split', 'float', 1e-4, 1e-2, {'log': True})
)

In [22]:
rf_best = RandomForestClassifier(**rf_op.model.get_params())
rf_best.fit(X_train, y_train)
rf_train_preds = rf_best.predict(X_train)
rf_test_preds = rf_best.predict(X_test)

In [23]:
print('Train:')
print(classification_report(y_train, rf_train_preds), '\n')
print('Test:')
print(classification_report(y_test, rf_test_preds))

Train:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88     14415
           1       0.80      0.71      0.75      7882

    accuracy                           0.84     22297
   macro avg       0.83      0.81      0.82     22297
weighted avg       0.83      0.84      0.83     22297
 

Test:
              precision    recall  f1-score   support

           0       0.83      0.88      0.85      4744
           1       0.77      0.67      0.71      2689

    accuracy                           0.81      7433
   macro avg       0.80      0.78      0.78      7433
weighted avg       0.80      0.81      0.80      7433



In [24]:
params_to_txt(rf_op)

## XGBoost

In [43]:
xgb_op = OptunaTuner(
    XGBClassifier, f1_score_average_weighted, 'maximize', # class-specific arguments
    seed=571, predictor='cpu_predictor', verbosity=0, nthread=7, # model-specific technical parameters
    objective='binary:hinge', eval_metric='error', n_estimators=1000 # model-specific fixed hyperparameters
)
xgb_op.fit(
    300, X_train, y_train, X_test, y_test,
    ('reg_alpha', 'loguniform', 1e-3, 10.0),
    ('reg_lambda', 'loguniform', 1e-3, 10.0),
    ('colsample_bytree', 'loguniform', 0.2, 1.0),
    ('subsample', 'loguniform', 0.4, 1.0),
    ('learning_rate', 'loguniform', 1e-4, 0.5),
    ('max_depth', 'categorical', [5, 10, 20, 30])
)

In [45]:
xgb_best = XGBClassifier(**xgb_op.model.get_params())
xgb_best.fit(X_train, y_train)
xgb_train_preds = xgb_best.predict(X_train)
xgb_test_preds = xgb_best.predict(X_test)

In [46]:
print('Train:')
print(classification_report(y_train, xgb_train_preds), '\n')
print('Test:')
print(classification_report(y_test, xgb_test_preds))

Train:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89     14415
           1       0.83      0.76      0.79      7882

    accuracy                           0.86     22297
   macro avg       0.85      0.84      0.84     22297
weighted avg       0.86      0.86      0.86     22297
 

Test:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      4744
           1       0.75      0.70      0.72      2689

    accuracy                           0.80      7433
   macro avg       0.79      0.78      0.79      7433
weighted avg       0.80      0.80      0.80      7433



In [47]:
params_to_txt(xgb_op)

## Identify overlaps of models errors

In [30]:
base_models_df = pd.DataFrame(
    {
        'svc_preds': svc_test_preds,
        'knn_preds': knn_test_preds,
        'rf_preds': rf_test_preds,
        'xgb_preds': xgb_test_preds,
        'y_true': y_test
    }
)

In [31]:
base_models_df['svc_accuracy'] = base_models_df.apply(lambda x: 'error' if x['svc_preds'] != x['y_true'] else 'correct', axis=1)
base_models_df['knn_accuracy'] = base_models_df.apply(lambda x: 'error' if x['knn_preds'] != x['y_true'] else 'correct', axis=1)
base_models_df['rf_accuracy'] = base_models_df.apply(lambda x: 'error' if x['rf_preds'] != x['y_true'] else 'correct', axis=1)
base_models_df['xgb_accuracy'] = base_models_df.apply(lambda x: 'error' if x['xgb_preds'] != x['y_true'] else 'correct', axis=1)

In [32]:
svc_errors = base_models_df[base_models_df['svc_accuracy']=='error'].index
knn_errors = base_models_df[base_models_df['knn_accuracy']=='error'].index
rf_errors = base_models_df[base_models_df['rf_accuracy']=='error'].index
xgb_errors = base_models_df[base_models_df['xgb_accuracy']=='error'].index

In [33]:
# Overlap of all models
len(
    set.intersection(
        set(svc_errors),
        set(knn_errors),
        set(rf_errors),
        set(xgb_errors)
    )
) / base_models_df.shape[0]

0.1442217139782053

In [39]:
len(
    set.intersection(
        set(rf_errors),
        set(xgb_errors)
    )
) / base_models_df.shape[0]

0.18269877572985335

## Models Stacking

In [40]:
stacking_model = StackingClassifier(
    estimators=[
        # ('svc', svc_best),
        # ('knn', knn_best),
        ('rf', rf_best),
        ('xgb', xgb_best)
    ],
    final_estimator=LogisticRegression(),
    cv=5
)

In [41]:
stacking_model.fit(X_train, y_train)
stacking_train_preds = stacking_model.predict(X_train)
stacking_test_preds = stacking_model.predict(X_test)

In [42]:
print('Train:')
print(classification_report(y_train, stacking_train_preds), '\n')
print('Test:')
print(classification_report(y_test, stacking_test_preds))

Train:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88     14415
           1       0.81      0.70      0.75      7882

    accuracy                           0.84     22297
   macro avg       0.83      0.81      0.81     22297
weighted avg       0.83      0.84      0.83     22297
 

Test:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      4744
           1       0.77      0.66      0.71      2689

    accuracy                           0.81      7433
   macro avg       0.80      0.77      0.78      7433
weighted avg       0.80      0.81      0.80      7433

